---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [81]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [4]:
#You can first read_csv with parameter name for create DataFrame with column Region Name, separator is value which is NOT in values (like ;):
df = pd.read_csv('university_towns.txt', sep=";", names=['RegionName'])
df.head()

,RegionName
0,Alabama[edit]
1,Auburn (Auburn University)[1]
2,Florence (University of North Alabama)
3,Jacksonville (Jacksonville State University)[2]
4,Livingston (University of West Alabama)[2]


In [5]:
#Then  insert new column State with extract rows where text [edit] and replace all values from ( to the end to column Region Name.
df.insert(0, 'State', df['RegionName'].str.extract('(.*)\[edit\]', expand=False).ffill())

df.head(10)


,State,RegionName
0,Alabama,Alabama[edit]
1,Alabama,Auburn (Auburn University)[1]
2,Alabama,Florence (University of North Alabama)
3,Alabama,Jacksonville (Jacksonville State University)[2]
4,Alabama,Livingston (University of West Alabama)[2]
5,Alabama,Montevallo (University of Montevallo)[2]
6,Alabama,Troy (Troy University)[2]
7,Alabama,"Tuscaloosa (University of Alabama, Stillman Co..."
8,Alabama,Tuskegee (Tuskegee University)[5]
9,Alaska,Alaska[edit]


In [6]:
df['RegionName'] = df['RegionName'].str.replace(r' \(.+$', '')
df.head()

,State,RegionName
0,Alabama,Alabama[edit]
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston


In [7]:
#Last remove rows where text [edit] by boolean indexing, mask is created by (~ used with string.contains)str.contains:
df = df[~df['RegionName'].str.contains('\[edit\]')].reset_index(drop=True)

df.head()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


In [8]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''    
    return df

get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [9]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls', 
                   skiprows = 220,
                   #index_col = 0,
                   names = ['Quarter', 'GDP'],
                   parse_cols = [4,6]                   
                  ) 
    #gdp.index.name = 'Quarter'
    for i in gdp.index:
        if (gdp.iloc[i+1][1] < gdp.iloc[i][1]) & (gdp.iloc[i+2][1] < gdp.iloc[i+1][1]):
        #if second_q < first_q) and (third_q < second_q) #then the first_q is the recession start. 
            return gdp.iloc[i+1][0]
get_recession_start()

'2008q3'

In [10]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    #second_q > first_q and third_q > second_q then third_q is the recession end.
    #Basically recession start is the presence of 2 consecutive gdp decline years and recession end is the presence of two continuous GDP increase years
    gdp = pd.read_excel('gdplev.xls', 
                   skiprows = 220,
                   names = ['Quarter', 'GDP'],
                   parse_cols = [4,6]                   
                  ) 
    for i in gdp.index:
        if (gdp.iloc[i+2][1] > (gdp.iloc[i][1] and gdp.iloc[i+1][1])) and  (gdp.iloc[i+1][1] > gdp.iloc[i][1]) and (gdp.iloc[i][0] > '2008q2'):
            return gdp.iloc[i+2][0]
        
get_recession_end()

'2009q4'

In [11]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls', 
                   skiprows = 220,
                   names = ['Quarter', 'GDP'],
                   parse_cols = [4,6]                   
                  ) 
    recession = gdp.loc[gdp['Quarter'].isin(['2008q2', '2008q3', '2008q4', '2009q1', '2009q2', '2009q3', '2009q4'])]
    recession_bottom = recession.loc[(recession['GDP'] == min(recession['GDP']))]
    return recession_bottom.iloc[0][0]
get_recession_bottom()

'2009q2'

In [12]:
homes = pd.read_csv('City_Zhvi_AllHomes.csv', usecols=[2, 1, *range(51, 251)])
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
homes['State'] = homes['State'].map(states)
homes.set_index(['State', 'RegionName'], inplace = True)#multiindexing
homes.head()

,,2000-01,2000-02,2000-03,2000-04,2000-05,2000-06,2000-07,2000-08,2000-09,2000-10,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
California,Los Angeles,204400.0,207000.0,209800.0,212300.0,214500.0,216600.0,219000.0,221100.0,222800.0,224300.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
Illinois,Chicago,136800.0,138300.0,140100.0,141900.0,143700.0,145300.0,146700.0,147900.0,149000.0,150400.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
Pennsylvania,Philadelphia,52700.0,53100.0,53200.0,53400.0,53700.0,53800.0,53800.0,54100.0,54500.0,54700.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
Arizona,Phoenix,111000.0,111700.0,112800.0,113700.0,114300.0,115100.0,115600.0,115900.0,116500.0,117200.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900


In [13]:
homes = homes.groupby(pd.PeriodIndex(homes.columns, freq='Q', inplace = True), axis=1).mean()#inplace = True, is required for querying 
homes.columns = homes.columns.strftime('%Yq%q') # convert Qs to 'q's

homes.shape#shows dimensions
homes.head()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0


In [14]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    return homes
#convert_housing_data_to_quarters()
convert_housing_data_to_quarters().loc["Texas"].loc["Austin"].loc["2010q3"]

202266.66666666666

In [57]:
#First create new data showing the decline or growth of housing prices between the recession start and the recession bottom.
homes = convert_housing_data_to_quarters()
#homes.head()
recession_prices = homes.iloc[:,33:38].copy()
recession_prices.head()

,,2008q2,2008q3,2008q4,2009q1,2009q2
State,RegionName,,,,,
New York,New York,503933.333333,499766.666667,487933.333333,477733.333333,465833.333333
California,Los Angeles,502266.666667,469500.000000,443966.666667,426266.666667,413900.000000
Illinois,Chicago,237900.000000,232000.000000,227033.333333,223766.666667,219700.000000
Pennsylvania,Philadelphia,118133.333333,116933.333333,115866.666667,116200.000000,116166.666667
Arizona,Phoenix,205266.666667,193766.666667,183333.333333,177566.666667,168233.333333


In [64]:
recession_prices['ratio'] = recession_prices['2008q2'] / recession_prices['2009q2']
#recession_prices.reset_index(inplace = True)
#recession_prices = recession_prices.replace('\n','', regex=True)
recession_prices.head()

,,2008q2,2008q3,2008q4,2009q1,2009q2,ratio
State,RegionName,,,,,,
New York,New York,503933.333333,499766.666667,487933.333333,477733.333333,465833.333333,1.081789
California,Los Angeles,502266.666667,469500.000000,443966.666667,426266.666667,413900.000000,1.213498
Illinois,Chicago,237900.000000,232000.000000,227033.333333,223766.666667,219700.000000,1.082840
Pennsylvania,Philadelphia,118133.333333,116933.333333,115866.666667,116200.000000,116166.666667,1.016930
Arizona,Phoenix,205266.666667,193766.666667,183333.333333,177566.666667,168233.333333,1.220131


In [68]:
#Then run a ttest comparing the university town values to the non-university towns values, 
#return whether the alternative hypothesis (that the two groups are the same)is true or not as well as the 
#p-value of the confidence.
univ_towns = get_list_of_university_towns()
univ_towns.set_index(['State', 'RegionName'], inplace = True)#multiindex
univ_towns.head()

Empty DataFrame
Columns: []
Index: [(Alabama, Auburn), (Alabama, Florence), (Alabama, Jacksonville), (Alabama, Livingston), (Alabama, Montevallo)]

In [77]:
#categorize data based on whether university town or not 
uni_gdp = recession_prices[recession_prices.index.isin(univ_towns.index)].dropna()
uni_gdp.shape 

(257, 6)

In [105]:
nonuni_gdp = recession_prices[~recession_prices.index.isin(univ_towns.index)].dropna()# ~ inverses the condition
nonuni_gdp.shape

(9599, 6)

In [110]:
#ttest
#from scipy.stats import ttest_ind
t_test = ttest_ind(uni_gdp['ratio'], nonuni_gdp['ratio'])
t_test

Ttest_indResult(statistic=-2.998032664179151, pvalue=0.0027240637047614541)

In [98]:
#Return the tuple (different, p, better)
#different=True if the t-test is True at a p<0.01 (we reject the null hypothesis)
different = p_value<0.01
uni_ratio = uni_gdp['ratio'].mean()
nonuni_ratio = nonuni_gdp['ratio'].mean()
def better():
    if uni_ratio < nonuni_ratio:
        return "university town"

different, p_value, better()

(True, 0.0027240637047614541, 'university town')

In [106]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    homes = convert_housing_data_to_quarters()
    recession_prices = homes.iloc[:,33:38].copy()
    recession_prices['ratio'] = recession_prices['2008q2'] / recession_prices['2009q2']
    univ_towns = get_list_of_university_towns()
    #univ_towns.set_index(['State', 'RegionName'], inplace = True) not needed here since already set
    uni_gdp = recession_prices[recession_prices.index.isin(univ_towns.index)].dropna()
    nonuni_gdp = recession_prices[~recession_prices.index.isin(univ_towns.index)].dropna()
    p_value = ttest_ind(uni_gdp['ratio'], nonuni_gdp['ratio'])[1]
    different = p_value<0.01
    uni_ratio = uni_gdp['ratio'].mean()
    nonuni_ratio = nonuni_gdp['ratio'].mean()
    def better():
        if uni_ratio < nonuni_ratio:
            return "university town" #tstat < 0, means university town performed better during recession
    return (different, p_value, better())
run_ttest()

(True, 0.0027240637047614541, 'university town')